<h1>SVM (Support Vector Machine)</h1>

Метод опорных векторов (далее SVM) - алгоритм машинного обучения, основной идеей которого является проекция данных в пространство большей размерности и нахождение разделяющей гиперплоскости с максимальным зазором. 

Задача классификации: основываясь на поминутных записях активности человека, определить, является ли он больным шизофренией.

<h3>Немного математики</h3>

Пускай обучающая выборка X состоит из n-мерных вещщественных (каждое значение $\in \mathbb{R}$) векторов, и для каждого такого вектора задана метка y = {-1, 1}. Множество меток для каждого вектора - Y. Тогда алгоритм классификации $a : X \to Y $

Построим линейный пороговый классификатор:

$$a(x) = sign\left(\sum_{j=1}^{n} w_j x^j - w_0 \right) = sign(\langle w, x \rangle - w_0) $$

Предположим, что выборка линейно-разделима, т.е. кол-во ошибок классификации 
$$\sum_{i=1}^{l} \left[y_i(\langle w, x\rangle - w_0)\right \lt 0]$$

равно нулю. Очевидно, что таких гиперплоскостей существует множество, и идея метода состоит в том, чтобы разумно использовать это свойство. Выберем пограничные точки (ближайшие к разделяющей гиперплоскости) следующим способом:
$$\langle w, x\rangle - w_0 = y_i$$

Ширина полосы, заданная неравенством, исходя из пограничных точек, является максимальной, когда норма $w$ минимальна.

Тогда задача сводится к задаче квадратичного программирования.

Для случая линейно неразделимой выборки используется классификация с мягким зазором, используя двойственные переменные или ядерный трюк (kernel trick), который использует функцию ядро для перевода $X \times X \to H$. 

Функция $K : X \times X \to H$ является ядром, если есть такая функция $\phi$, такая что $K(x, x^{'}) =  \ \langle \phi(x), \phi(x^{'} \rangle$. Также существует теорема Мерсера (1909), которая описывает необходимое условие для того, чтобы функция являлась ядром.

Реализуем SVM с помощью sklearn.

<h2>Настраиваем генератор данных</h2>

Для начала надо реализовать подачу данных для модели. В Python существует функция-генератор, которую мы и будем использовать.

In [1]:
import os

import pandas as pd
import numpy as np

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer

In [2]:
def gen_data(path:str):
    '''Function, that generates data from path\control and path\patient.'''
    
    # generating control objects
    for file in os.listdir(os.path.join(path, 'control')):
    
        # building full-name of file
        full_file = os.path.join(path, os.path.join('control', file))
        
        # reading .csv file
        df = pd.read_csv(full_file)
        
        # generating activity of object and label
        yield df['activity'], 0
        
    
    # generating patient objects
    for file in os.listdir(os.path.join(path, 'patient')):
    
        # building full-name of file
        full_file = os.path.join(path, os.path.join('patient', file))
        
        # reading .csv file
        df = pd.read_csv(full_file)
        
        # generating activity of object and label
        yield df['activity'].values, 1

Т. к. у нас обучающая выборка маленькая, то мы можем загрузить все данные в RAM.

In [3]:
data = [pair for pair in gen_data('data')]

data[:3]

[(0          30
  1         130
  2          70
  3           3
  4           3
  5           3
  6           3
  7           3
  8           3
  9           3
  10          3
  11          3
  12          3
  13          3
  14          3
  15          3
  16          3
  17          3
  18          3
  19          3
  20          3
  21          3
  22          3
  23          3
  24          3
  25          3
  26          3
  27          3
  28          3
  29          3
           ... 
  20130     809
  20131     835
  20132     835
  20133     587
  20134     386
  20135     689
  20136     646
  20137     646
  20138     667
  20139     172
  20140     919
  20141     286
  20142     120
  20143     338
  20144     550
  20145     835
  20146     689
  20147     268
  20148     166
  20149     689
  20150     373
  20151     204
  20152     411
  20153     317
  20154     439
  20155     485
  20156     398
  20157     646
  20158    1474
  20159    1474
  Name: activity, Length

Теперь разделим данные на X и Y.

In [4]:
# X set with values of acitivity
X = np.array([pair[0] for pair in data])

# Y set with labels
Y = np.array([pair[1] for pair in data])

# we don't need this list 
del data

X[:3], Y[:3]

(array([[  30,  130,   70, ...,  646, 1474, 1474],
        [   5,    3,    3, ...,  172,  500,   79],
        [   3,    2,    0, ...,    0,    0,    0]], dtype=int64),
 array([0, 0, 0]))

С помощью функции train_test_split разделим данные на тренировочные и тестовые в пропорции 17:3. 

In [5]:
# splitting data to train/test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, shuffle=True, test_size=0.15)

len(X_train), len(X_test)

(31, 6)

Создадим модель машинного обучения и оценим её точность на тренировочных и тестовых данных.

In [6]:
# creating model
model = svm.SVC(gamma='auto')

# fitting
model.fit(X_train, Y_train)

# predictions for test data
test_predicted = model.predict(X_test)

# predictions for train data
train_predicted = model.predict(X_train)

# output
print(f'Test accuracy score is {accuracy_score(test_predicted, Y_test)}')
print(f'Train accuracy score is {accuracy_score(train_predicted, Y_train)}')

Test accuracy score is 0.3333333333333333
Train accuracy score is 1.0


Как мы видим, модель столкнулась с переобучением. Попробуем изменить параметр С, который отвечает за регуляризацию.

In [7]:
# creating model
model = svm.SVC(gamma='auto', C=0.1)

# fitting
model.fit(X_train, Y_train)

# predictions for test data
test_predicted = model.predict(X_test)

# predictions for train data
train_predicted = model.predict(X_train)

print(f'Test accuracy score is {accuracy_score(test_predicted, Y_test)}')
print(f'Train accuracy score is {accuracy_score(train_predicted, Y_train)}')

Test accuracy score is 0.3333333333333333
Train accuracy score is 0.5806451612903226


Стало ещё хуже, однако в наших данных есть одна проблема. Они не приведены к общему диапазону, т.е. признак с большим значением является более важным, чем тот, который имеет маленькое значение, однако это неправильно. Для этого мы создадим конвейер, который будет приводить  нормализировать данные, а затем выполнять предсказывание или обучение модели.

In [8]:
# creating normalizer object 
normalizer = Normalizer()

# creating SVC model
model = svm.SVC(gamma='auto', C=0.1)

# creating pipeline
steps = [('normalizer', normalizer), ('classifier', model)]
pipeline = Pipeline(steps)

In [9]:
# fitting
pipeline.fit(X_train, Y_train)

# predictions for test data
test_predicted = pipeline.predict(X_test)

# predictions for train data
train_predicted = pipeline.predict(X_train)

print(f'Test accuracy score is {accuracy_score(test_predicted, Y_test)}')
print(f'Train accuracy score is {accuracy_score(train_predicted, Y_train)}')

Test accuracy score is 0.3333333333333333
Train accuracy score is 0.5806451612903226


Опять мы не получаем нужной "мощности" модели. Предположительно, это связанно с тем, что модель не учитывает разницу в активности за какой-то промежуток. И это я собираюсь исправить с помощью рекуррентной нейронной сети, о чём смотрите в LSTM.ipynb.